In [3]:
!pip install keras-bert

  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=b2e241de581449ecfa44eb63236b5cd907be150ff1c483114d137c67fe1bd85b
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for keras-transformer: filename=keras_transformer-0.38.0-cp36-none-any.whl size=12942 sha256=2362cdce9cf4a9c69c3b5672e057e1d9cd1e8918ce994cef9815f310d682a380
  Stored in directory: /root/.cache/pip/wheels/e5/fb/3a/37b2b9326c799aa010ae46a04ddb04f320d8c77c0b7e837f4e
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=0f91aa8f4bf9f85420a2488593d45ba7b6c1f05dc81449c73f2b3d495394c9e3
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp36-none-any.whl size=15612 sha256=fddcb10ea46ab5a260dbe7094491f09c3df91f1b295612a23fbc4b84897b2677
  Stor

In [5]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 6.2MB/s 


In [9]:
!pip install emoji --upgrade

     |████████████████████████████████| 51kB 7.1MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=d8b31c7009662e32223d2b5ca5a30bd667bfa3e11e22f3450790c197030ead2b
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [7]:
%cd /content/drive/MyDrive/Colab\ Notebooks/cs410/CourseProject  #cd to folder which contain required external files, TEXT_MODEL.py and text_preprocessing.py


/content/drive/MyDrive/Colab Notebooks/cs410/CourseProject


In [10]:
import tensorflow as tf 
import tensorflow_hub as hub 
from tensorflow.keras import layers
import bert
import numpy as np 
import pandas as pd 
import json
import re
import random
import math
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from TEXT_MODEL import TEXT_MODEL
from TEXT_PREPROCESSING import preprocess_text


In [12]:
# LOADING DATA
categorized_tweets = pd.read_json('./data/train.jsonl', lines = True)
categorized_tweets.isnull().values.any()
print(categorized_tweets)

            label  ...                                            context
0         SARCASM  ...  [A minor child deserves privacy and should be ...
1         SARCASM  ...  [@USER @USER Why is he a loser ? He's just a P...
2         SARCASM  ...  [Donald J . Trump is guilty as charged . The e...
3         SARCASM  ...  [Jamie Raskin tanked Doug Collins . Collins lo...
4         SARCASM  ...  [Man ... y ’ all gone “ both sides ” the apoca...
...           ...  ...                                                ...
4995  NOT_SARCASM  ...  [@USER Apologies for the inconvenience you fac...
4996  NOT_SARCASM  ...  [@USER 🤔 idk tho , I think I ’ m #hungry . But...
4997  NOT_SARCASM  ...  [@USER @USER @USER Peace to you , and two coun...
4998  NOT_SARCASM  ...  [Bernie Sanders told Elizabeth Warren in priva...
4999  NOT_SARCASM  ...  [PDP PROTEST BRAINSTORMING SESSION Deji : We n...

[5000 rows x 3 columns]


In [13]:
# PREPROCESSING DATA
tweets = []
data = list(categorized_tweets["response"])
print(data[0])
for d in data:
    tweets.append(preprocess_text(d))

y = categorized_tweets["label"]
y = np.array(list(map(lambda x: 1 if x=="SARCASM" else 0, y)))

@USER @USER @USER I don't get this .. obviously you do care or you would've moved right along .. instead you decided to care and troll her ..


In [15]:
# TOKENIZING DATA
num_words = 20000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer.fit_on_texts(tweets)
train_sequences = tokenizer.texts_to_sequences(tweets)
maxlen = max([len(x) for x in train_sequences])
tokenized_tweets = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

# tokenized example
print(tweets[9])
print(tokenized_tweets[9])

@USER @USER @USER responds to facts by tossing out frantic insults , then accuses others of being " triggered by facts " :rolling_on_the_floor_laughing: :face_with_tears_of_joy: :rolling_on_the_floor_laughing:
[   2    2    2 6083    4  443   50 6084   57 6085 1050  102 2510  234
    8   87 1347   50  443  131   17    3  158  148   31   15  117    8
   84  131   17    3  158  148    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


In [16]:
# PREPARE FOR TRAINING
tweets_with_len = [[tweet, y[i], len(tweet)] for i, tweet in enumerate(tokenized_tweets)]
random.shuffle(tweets_with_len)
tweets_with_len.sort(key=lambda x: x[2])
sorted_tweet_labels = [(tweet_lab[0], tweet_lab[1]) for tweet_lab in tweets_with_len]
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_tweet_labels, output_types=(tf.int32, tf.int32))

BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))
next(iter(batched_dataset))
TOTAL_BATCHES = math.ceil(len(sorted_tweet_labels) / BATCH_SIZE)
batched_dataset.shuffle(TOTAL_BATCHES)

<ShuffleDataset shapes: ((None, None), (None,)), types: (tf.int32, tf.int32)>

In [17]:
# BUILD MODEL
# Model hyperparameters
VOCAB_LENGTH = num_words
EMB_DIM = 200
CNN_FILTERS = 200
DNN_UNITS = 512
OUTPUT_CLASSES = 2
DROPOUT_RATE = 0.2
NB_EPOCHS = 10

# Model object
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [18]:
# Compile model
text_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Fit model
text_model.fit(batched_dataset, epochs=NB_EPOCHS)

Epoch 1/10
157/157 [==============================] - 7s 44ms/step - loss: 0.5894 - accuracy: 0.6682
Epoch 2/10
157/157 [==============================] - 7s 43ms/step - loss: 0.2683 - accuracy: 0.8884
Epoch 3/10
157/157 [==============================] - 7s 43ms/step - loss: 0.0489 - accuracy: 0.9836
Epoch 4/10
157/157 [==============================] - 7s 43ms/step - loss: 0.0048 - accuracy: 0.9992
Epoch 5/10
157/157 [==============================] - 7s 43ms/step - loss: 0.0033 - accuracy: 0.9996
Epoch 6/10
157/157 [==============================] - 7s 43ms/step - loss: 0.0019 - accuracy: 0.9998
Epoch 7/10
157/157 [==============================] - 7s 43ms/step - loss: 0.0021 - accuracy: 0.9998
Epoch 8/10
157/157 [==============================] - 7s 43ms/step - loss: 0.0012 - accuracy: 0.9998
Epoch 9/10
157/157 [==============================] - 7s 44ms/step - loss: 0.0024 - accuracy: 0.9998
Epoch 10/10
157/157 [==============================] - 7s 43ms/step - loss: 7.8772e-04 - ac

In [20]:

# Predict using model
uncat_tweets = pd.read_json('./data/test.jsonl', lines = True)
un_tweets = []
uncat_data = list(uncat_tweets["response"])

for d in uncat_data:
    un_tweets.append(preprocess_text(d))

test_sequences = tokenizer.texts_to_sequences(un_tweets)
tokenized_un_tweets = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

processed_dataset = tf.data.Dataset.from_generator(lambda:tokenized_un_tweets, output_types=tf.int32)
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=(None, ))

predictions = text_model.predict(batched_dataset)

with open('answer.txt', 'w') as f:
    c = 1
    s_c = 0
    ns_c = 0
    for p in predictions:
        if p >= .5:
            f.write("twitter_" + str(c) + "," + "SARCASM\n")
            c += 1
            s_c += 1
        else:
            f.write("twitter_" + str(c) + "," + "NOT_SARCASM\n")
            c += 1
            ns_c += 1
print("# sarcasm: " + str(s_c))
print("# not sarcasm: " + str(ns_c))

# sarcasm: 925
# not sarcasm: 875


In [21]:
!git pull


Already up to date.
